In [ ]:
import sys
import cv2
import mediapipe as mp
import numpy as np
from PyQt6.QtWidgets import QApplication, QLabel, QPushButton, QVBoxLayout, QWidget
from PyQt6.QtGui import QImage, QPixmap
from PyQt6.QtCore import QTimer

In [ ]:
class FaceDetectionApp(QWidget):
    def __init__(self):
        super().__init__()

        # Configuration de l'interface
        self.setWindowTitle("Détection de Visage - App Moderne")
        self.setGeometry(100, 100, 800, 600)

        # Layout principal
        self.layout = QVBoxLayout()

        # Label pour afficher la vidéo
        self.video_label = QLabel(self)
        self.layout.addWidget(self.video_label)

        # Bouton Quitter
        self.quit_button = QPushButton("Quitter")
        self.quit_button.clicked.connect(self.close_app)
        self.layout.addWidget(self.quit_button)

        self.setLayout(self.layout)

        # Initialisation de la webcam
        self.cap = cv2.VideoCapture(0)

        # Initialisation de Mediapipe
        self.mp_face_detection = mp.solutions.face_detection
        self.face_detection = self.mp_face_detection.FaceDetection(min_detection_confidence=0.5)

        # Timer pour la mise à jour de la vidéo
        self.timer = QTimer()
        self.timer.timeout.connect(self.update_frame)
        self.timer.start(30)  # 30ms pour un affichage fluide

    def update_frame(self):
        ret, frame = self.cap.read()
        if not ret:
            return

        # Conversion pour Mediapipe (BGR → RGB)
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = self.face_detection.process(frame_rgb)

        # Dessiner les boîtes des visages détectés
        if results.detections:
            for detection in results.detections:
                bboxC = detection.location_data.relative_bounding_box
                h, w, _ = frame.shape
                x, y, w_box, h_box = int(bboxC.xmin * w), int(bboxC.ymin * h), int(bboxC.width * w), int(bboxC.height * h)
                cv2.rectangle(frame, (x, y), (x + w_box, y + h_box), (0, 255, 0), 3)

        # Convertir l'image pour PyQt
        height, width, channel = frame.shape
        bytes_per_line = 3 * width
        qt_img = QImage(frame.data, width, height, bytes_per_line, QImage.Format.Format_RGB888)
        self.video_label.setPixmap(QPixmap.fromImage(qt_img))

    def close_app(self):
        self.cap.release()
        self.timer.stop()
        self.close()

# Lancer l'application
if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = FaceDetectionApp()
    window.show()
    sys.exit(app.exec())
